# Minimal example - Pytorch API

## Define nn.Module with feature extractor network

Notes:
* In the forward funcition one must define the `self.feature_maps = [some_tensor, another_tensor]` parameter. This parameter is used by the `PytorchReceptiveField` to get the desired feature maps. 
* The feature_maps is a list of tensors of shape [batch_size, num_channels, height, width].

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn

class Linear(nn.Module):
    """An identity activation function"""
    def forward(self, x):
        return x
    

class SimpleVGG(nn.Module):
    def __init__(self, disable_activations: bool = False):
        """disable_activations: whether to generate network with Relus or not."""
        super(SimpleVGG, self).__init__()
        self.features = self._make_layers(disable_activations)

    def forward(self, x):
        # index of layers with feature maps
        select = [8, 13]
        # self.feature_maps is a list of Tensors, PytorchReceptiveField looks for 
        # this parameter and compute receptive fields for Tensors in self.feature_maps.
        self.feature_maps = []
        for l, layer in enumerate(self.features):
            x = layer(x)
            if l in select:
                self.feature_maps.append(x)
        return x

    def _make_layers(self, disable_activations: bool):
        activation = lambda: Linear() if disable_activations else nn.ReLU()
        layers = [
            nn.Conv2d(3, 64, kernel_size=3),
            activation(),
            nn.Conv2d(64, 64, kernel_size=3),
            activation(),
            
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3),
            activation(),
            nn.Conv2d(128, 128, kernel_size=3),
            activation(), # 8
            
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3),
            activation(),
            nn.Conv2d(256, 256, kernel_size=3),
            activation(), # 13
        ]        
        return nn.Sequential(*layers)    

## Define model build function
Notes:
* In the case when using layers which behave differently on training and eval modes, it is recommended to set model to an eval mode: `model.eval()`

In [3]:
def model_fn() -> nn.Module:
    model = SimpleVGG(disable_activations=True)
    model.eval()
    return model

## Run RF estimation

In [4]:
from receptivefield.pytorch import PytorchReceptiveField

input_shape = [96, 96, 3]
rf = PytorchReceptiveField(model_fn)
rf_params = rf.compute(input_shape = input_shape) 

ModuleNotFoundError: No module named 'receptivefield'

In [ ]:
rf.feature_maps_desc

In [ ]:
from receptivefield.image import get_default_image
rf.plot_rf_grids(get_default_image(input_shape, name='cat'), figsize=(20, 12), layout=(1, 2))

In [ ]:
rf.plot_gradient_at(fm_id=1, point=(9, 9))